In [8]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv('titanic.csv')

In [140]:
# Create an in-memory SQLite database
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [141]:
df.to_sql('titanic', con=engine)

In [143]:
engine.execute("SELECT * FROM titanic Limit 5").fetchall()

[(0, 0, 3, 'Mr. Owen Harris Braund', 'male', 22.0, 1, 0, 7.25),
 (1, 1, 1, 'Mrs. John Bradley (Florence Briggs Thayer) Cumings', 'female', 38.0, 1, 0, 71.2833),
 (2, 1, 3, 'Miss. Laina Heikkinen', 'female', 26.0, 0, 0, 7.925),
 (3, 1, 1, 'Mrs. Jacques Heath (Lily May Peel) Futrelle', 'female', 35.0, 1, 0, 53.1),
 (4, 0, 3, 'Mr. William Henry Allen', 'male', 35.0, 0, 0, 8.05)]

In [19]:
df.Name = df.Name.str.replace("'",'')

In [20]:
df.Name

0                                 Mr. Owen Harris Braund
1      Mrs. John Bradley (Florence Briggs Thayer) Cum...
2                                  Miss. Laina Heikkinen
3            Mrs. Jacques Heath (Lily May Peel) Futrelle
4                                Mr. William Henry Allen
5                                        Mr. James Moran
6                                 Mr. Timothy J McCarthy
7                          Master. Gosta Leonard Palsson
8       Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson
9                     Mrs. Nicholas (Adele Achem) Nasser
10                        Miss. Marguerite Rut Sandstrom
11                               Miss. Elizabeth Bonnell
12                         Mr. William Henry Saundercock
13                            Mr. Anders Johan Andersson
14                   Miss. Hulda Amanda Adolfina Vestrom
15                        Mrs. (Mary D Kingcome) Hewlett
16                                   Master. Eugene Rice
17                           Mr

In [20]:
len(df.columns)

8

In [28]:
convert_types ={'int64':'int', 'object': 'text', 'float64': 'real'}

In [52]:
col_types = pd.DataFrame(df.dtypes, columns=['types'])
col_types = col_types.replace(convert_types)
col_types

,types
Survived,int
Pclass,int
Name,text
Sex,text
Age,real
Siblings/Spouses Aboard,int
Parents/Children Aboard,int
Fare,real


In [48]:
col_types.types.values

array(['int', 'int', 'text', 'text', 'real', 'int', 'int', 'real'],
      dtype=object)

In [70]:
col_n_type = []
for i, j in zip(col_types.index, col_types.types.values):
    result = i + ' ' + j
    col_n_type.append(result)
    
col_n_type

['Survived int',
 'Pclass int',
 'Name text',
 'Sex text',
 'Age real',
 'Siblings/Spouses Aboard int',
 'Parents/Children Aboard int',
 'Fare real']

In [71]:
col_n_type = ','.join(col_n_type).replace('/', '_').replace(' Aboard', '_Aboard')
col_n_type

'Survived int,Pclass int,Name text,Sex text,Age real,Siblings_Spouses_Aboard int,Parents_Children_Aboard int,Fare real'

In [55]:
def type_func(table_name, df):
    
    # dataframe of columns and types
    col_types = pd.DataFrame(df.dtypes, columns=['types'])
    col_types = col_types.replace(convert_types)
    
    # Creating string of column names and type for creating SQL table
    col_n_type = []
    for i, j in zip(col_types.index, col_types.types.values):
        result = i + ' ' + j
        col_n_type.append(result)
    
    # Converting into one string
    col_n_type = ','.join(col_n_type)
    
    # Creating SQL table
    create = f"CREATE TABLE {table_name}({col_n_type});"
    
    return create

In [56]:
type_func('table', df=df)

'CREATE TABLE table(Survived int,Pclass int,Name text,Sex text,Age real,Siblings/Spouses Aboard int,Parents/Children Aboard int,Fare real);'

In [57]:
','.join(df.columns)

'Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare'

# `copy_table` Function

In [27]:
import psycopg2 as pg
pg.connect


import pandas as pd
import psycopg2 as pg


# Posgresql database parameters
dbname ='hartabvb'
user = 'hartabvb'
password = '9jk1ptYGv0qclPjUQybvhF8bnmAXPmNv'
host = 'stampy.db.elephantsql.com'

pg_conn = pg.connect(dbname=dbname, user=user, password=password, host=host)

In [28]:
def copy_table(table_name, csv_file):
    import pandas as pd

    df = pd.read_csv(csv_file)
    #df.Name = df.Name.str.replace('[^\w\s]','')
    df.Name = df.Name.str.replace("'",'')
    # Cursor
    pg_curs = pg_conn.cursor()
    
    # dataframe of columns and types
    convert_types ={'int64':'int', 'object': 'text', 'float64': 'real'}
    
    col_types = pd.DataFrame(df.dtypes, columns=['types'])
    col_types = col_types.replace(convert_types)
    
    # Creating string of column names and type for creating SQL table
    col_n_type = []
    for i, j in zip(col_types.index, col_types.types.values):
        result = i + ' ' + j
        col_n_type.append(result)
    
    # Converting into one string
    col_n_type = ','.join(col_n_type).replace('/', '_').replace(' Aboard', '_Aboard')
    
    # Creating SQL table
    create = f"CREATE TABLE {table_name}({col_n_type});"
    pg_curs.execute(create)

    # Now we insert the data in the empty table created
    col_names = ','.join(df.columns).replace('/', '_').replace(' Aboard', '_Aboard')
    rows = df.values
    
    for row in [tuple(i) for i in rows]:
        insert_result = f"INSERT INTO {table_name} ({col_names}) VALUES" + str(row)
        pg_curs.execute(insert_result)


    pg_conn.commit()






In [29]:
copy_table('titanic_table2', 'titanic.csv')